In [141]:
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
import pylab as pl
import numpy as np

import warnings                              # Disable some warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [143]:
import pandas as pd

#df = pd.read_parquet("part-00001-9abd7f37-bb22-4458-b465-f0d345f512bf-c000.snappy.parquet")
#df_2 = pd.read_parquet("part-00000-7706f9e9-9322-4361-a006-85018a7ed901-c000.snappy.parquet")
df_1= pd.read_parquet("part-00000-c39f1c5c-b6aa-4659-8dd9-965ccc023d43-c000.snappy.parquet")
df_2 =pd.read_parquet("part-00001-912fc2f1-c131-4414-b9d8-bd9d4095ada5-c000.snappy.parquet")
df = pd.concat([df_1, df_2], axis=0, ignore_index=True)
df.tail()

,text,date,sentiment_score,btc_current_price,btc_price_24h_later,price_change_24h_in_%
781,Kelly Cooper is a fantastic America First Cand...,2024-10-30 14:25:00,0.9750,72161.0,NaN,NaN
782,Derek Merrin is a fantastic America First Cand...,2024-10-30 14:31:00,0.9818,72133.0,NaN,NaN
783,Congresswoman Carol Miller is a Fantastic Advo...,2024-10-30 14:31:00,0.9614,72133.0,NaN,NaN
784,"""RT @TeamTrumpPRESIDENT TRUMP: IN CONCLUSION",2024-11-03 18:15:00,0.0000,68333.0,NaN,NaN
785,"""KAMALA BROKE IT, BUT I WILL FIX IT! With your...",2024-11-03 23:32:00,0.9760,68752.0,NaN,NaN


In [145]:
df.isnull().sum()

text                      0
date                      0
sentiment_score           0
btc_current_price         0
btc_price_24h_later      27
price_change_24h_in_%    27
dtype: int64

In [147]:
df.dropna(inplace=True)

In [149]:
df["btc_price_increased"]=np.where(df["btc_price_24h_later"]-df["btc_current_price"]>0,1,0)
df_copy=df.copy()
df_copy

,text,date,sentiment_score,btc_current_price,btc_price_24h_later,price_change_24h_in_%,btc_price_increased
0,The recession was made worse by @BarackObama. ...,2012-02-15 21:09:00,-0.8752,4.98,4.59,-7.83,0
1,America is mired in the longest job recession ...,2012-03-19 18:36:00,-0.3400,5.19,4.97,-4.24,0
2,White House Press Sec. had a hard time explain...,2012-03-30 18:24:00,0.2263,4.91,4.89,-0.41,0
3,"By @BarackObama's design, the middle class wil...",2012-04-18 18:52:00,0.0000,5.14,5.13,-0.19,0
4,"""Selective memory - @BarackObama says that he ...",2012-05-11 17:55:00,-0.1779,4.93,4.97,0.81,1
...,...,...,...,...,...,...,...
764,"""https://www.coindesk.com/policy/2024/08/30/cr...",2024-09-12 17:24:00,0.0000,57679.00,59617.00,3.36,1
765,"When a prizefighter loses a fight, the first w...",2024-09-12 19:08:00,-0.6588,57733.00,59807.00,3.59,1
766,BATTLEGROUND STATE INFLATION🚀,2024-09-13 02:14:00,-0.5319,57952.00,60528.00,4.45,1
767,"""RT @realDonaldTrumpBATTLEGROUND STATE INFLATI...",2024-09-13 03:03:00,0.0000,58136.00,60378.00,3.86,1


In [151]:
df["sentiment_score"].abs().mean()

0.586623978919631

In [153]:
#columns_to_keep=["sentiment_score","btc_current_price","btc_price_increased"]
columns_to_keep=["sentiment_score","btc_current_price","price_change_24h_in_%"]
df=df[columns_to_keep]
df

,sentiment_score,btc_current_price,price_change_24h_in_%
0,-0.8752,4.98,-7.83
1,-0.3400,5.19,-4.24
2,0.2263,4.91,-0.41
3,0.0000,5.14,-0.19
4,-0.1779,4.93,0.81
...,...,...,...
764,0.0000,57679.00,3.36
765,-0.6588,57733.00,3.59
766,-0.5319,57952.00,4.45
767,0.0000,58136.00,3.86


In [155]:
from sklearn.preprocessing import StandardScaler

# Selecting features for anomaly detection
#features = ['sentiment_score', 'btc_current_price', 'btc_price_24h_later']
#features = ['sentiment_score', 'btc_current_price', 'btc_price_24h_later',"price_change_24h_in_%"]
#features = ['sentiment_score',"price_change_24h_in_%"]

# Standardize data
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

In [157]:
from sklearn.ensemble import IsolationForest

# Initialize the Isolation Forest model
#model = IsolationForest(contamination=0.05, random_state=42)  # Assuming 5% anomalies
model = IsolationForest(contamination=0.1, random_state=42)  # Assuming 5% anomalies
# Fit model and predict anomalies (-1 = anomaly, 1 = normal)
df['anomaly'] = model.fit_predict(df_scaled)

# Count anomalies
print(df['anomaly'].value_counts())


anomaly
 1    683
-1     76
Name: count, dtype: int64


In [159]:
df_copy["anomaly"]=df['anomaly']

In [161]:
df_copy[df_copy['anomaly'] == -1]["sentiment_score"].abs().mean()

0.6478552631578947

In [163]:
df_copy[df_copy['anomaly'] == 1]["sentiment_score"].abs().mean()

0.579810541727672

In [165]:
df_copy[df_copy['anomaly'] == -1]["price_change_24h_in_%"].abs().mean()

7.065789473684211

In [167]:
df_copy[df_copy['anomaly'] == -1]["price_change_24h_in_%"].mean()

1.9192105263157893

In [169]:
df_copy[df_copy['anomaly'] == 1]["price_change_24h_in_%"].abs().mean()

1.7505563689604686

In [171]:
df_copy[df_copy['anomaly'] == -1].groupby("btc_price_increased").count()

,text,date,sentiment_score,btc_current_price,btc_price_24h_later,price_change_24h_in_%,anomaly
btc_price_increased,,,,,,,
0,30,30,30,30,30,30,30
1,46,46,46,46,46,46,46


In [173]:
df_copy[df_copy['anomaly'] == 1].groupby("btc_price_increased").count()

,text,date,sentiment_score,btc_current_price,btc_price_24h_later,price_change_24h_in_%,anomaly
btc_price_increased,,,,,,,
0,297,297,297,297,297,297,297
1,386,386,386,386,386,386,386


In [175]:
anomalies=df_copy[df_copy['anomaly'] == -1]
anomalies

,text,date,sentiment_score,btc_current_price,btc_price_24h_later,price_change_24h_in_%,btc_price_increased,anomaly
0,The recession was made worse by @BarackObama. ...,2012-02-15 21:09:00,-0.8752,4.98,4.59,-7.83,0,-1
29,"With the impending crisis in Korea, is it a bi...",2013-04-02 19:46:00,-0.0772,112.08,132.79,18.48,1,-1
34,Sec of State Kerry said we would not go back t...,2014-01-06 16:38:00,-0.8096,960.33,899.14,-6.37,0,-1
51,RT @realDonaldTrump: ...big unnecessary regula...,2018-01-16 00:52:00,-0.2960,13890.92,11688.18,-15.86,0,-1
59,RT @The_Trump_Train: If GM doesn't want to kee...,2018-11-28 01:40:00,0.1969,3713.85,4330.71,16.61,1,-1
...,...,...,...,...,...,...,...,...
739,"""https://justthenews.com/government/security/j...",2024-08-08 14:25:00,0.0000,57475.00,60444.00,5.17,1,-1
740,State Senator Ileana Garcia is a Tireless Advo...,2024-08-11 04:40:00,0.6221,61143.00,58380.00,-4.52,0,-1
752,MASSIVE SCANDAL! The Harris-Biden Administrati...,2024-08-21 15:39:00,-0.9024,59402.00,61266.00,3.14,1,-1
753,"She talks about Compassion, but doesn’t talk a...",2024-08-23 03:11:00,-0.9854,60317.00,64185.00,6.41,1,-1


In [177]:
anomalies.to_excel("analyse.xlsx")

In [179]:
import pandas as pd
import re
from collections import Counter
from nltk.corpus import stopwords
import nltk

# Ensure stopwords are downloaded
nltk.download('stopwords')

# Load your dataframe (assuming df is already loaded)
text_data = " ".join(anomalies["text"])  # Combine all text into one large string

# Tokenization: Remove special characters and split words
words = re.findall(r'\b\w+\b', text_data.lower())  # Convert to lowercase

# Remove stopwords (common words that don't add meaning)
stop_words = set(stopwords.words('english'))
filtered_words = [word for word in words if word not in stop_words]

# Count word frequencies
word_counts = Counter(filtered_words)

# Get top 5 most common words
top_5_words = word_counts.most_common(15)

# Display results
print("Top 5 Most Repetitive Words:")
for word, count in top_5_words:
    print(f"{word}: {count}")


Top 5 Most Repetitive Words:
inflation: 45
biden: 33
great: 27
country: 24
never: 24
joe: 21
economy: 21
people: 21
trump: 19
america: 19
crooked: 19
amp: 17
border: 17
crime: 17
president: 15


[nltk_data] Downloading package stopwords to /Users/diego/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [192]:
import pandas as pd
import re
from collections import Counter
from nltk.corpus import stopwords
import nltk

# Ensure stopwords are downloaded
nltk.download('stopwords')

# Load your dataframe (assuming df is already loaded)
stop_words = set(stopwords.words('english'))

# Dictionary to store word occurrence across unique tweets
word_tweet_count = Counter()

# Iterate through each tweet
for text in anomalies["text"]:
    words = set(re.findall(r'\b\w+\b', text.lower()))  # Convert to lowercase and extract unique words per tweet
    filtered_words = {word for word in words if word not in stop_words}  # Remove stopwords
    word_tweet_count.update(filtered_words)  # Count in how many tweets each word appears

# Get top 15 words appearing in most unique tweets
top_15_words_in_tweets = word_tweet_count.most_common(15)

# Display results
print("Top 15 Words Appearing in Most Unique Tweets:")
for word, count in top_15_words_in_tweets:
    print(f"{word}: {count}")


Top 15 Words Appearing in Most Unique Tweets:
inflation: 42
great: 22
economy: 20
biden: 19
never: 18
people: 18
country: 17
president: 15
america: 15
border: 15
joe: 15
borders: 14
amp: 13
crooked: 13
crime: 13


[nltk_data] Downloading package stopwords to /Users/diego/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [194]:
import pandas as pd
import re
from collections import Counter
from nltk.corpus import stopwords
import nltk

# Ensure stopwords are downloaded
nltk.download('stopwords')

# Combine all text into one large string
text_data = " ".join(anomalies["text"])

# Tokenization: Remove special characters and split words
words = re.findall(r'\b\w+\b', text_data.lower())

# Remove stopwords (common words like "the", "is", etc.)
stop_words = set(stopwords.words('english'))
filtered_words = [word for word in words if word not in stop_words]

# Count word frequencies
word_counts = Counter(filtered_words)

# Get the most common word
most_common_word, _ = word_counts.most_common(1)[0]
most_common_word="biden"

# Print rows where the most common word appears in the "text" column
filtered_df = anomalies[anomalies["text"].str.contains(most_common_word, case=False, na=False)]

# Display the result
filtered_df


[nltk_data] Downloading package stopwords to /Users/diego/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text,date,sentiment_score,btc_current_price,btc_price_24h_later,price_change_24h_in_%,btc_price_increased,anomaly
231,The show “60 Minutes” should be ashamed of the...,2023-10-16 00:29:00,-0.9622,27064.0,28612.0,5.72,1,-1
233,"The Failing New York Times story, leaked by De...",2023-10-23 02:44:00,0.8047,29984.0,33955.0,13.24,1,-1
240,"Merry Christmas to all, including Crooked Joe ...",2023-12-25 19:38:00,-0.9344,43621.0,42694.0,-2.13,0,-1
264,"Jeff Zeleny, CNN Chief National Affairs Corres...",2024-05-14 18:52:00,-0.8999,61607.0,64422.0,4.57,1,-1
265,"""RT @realDonaldTrumpCrooked Joe Biden is the W...",2024-05-15 16:41:00,-0.8384,63492.0,66179.0,4.23,1,-1
267,Congressman James Comer is an incredible Repre...,2024-05-20 00:48:00,0.4186,66178.0,69378.0,4.84,1,-1
270,South Carolina Speaker of the House Murrell Sm...,2024-05-20 03:41:00,0.9773,66255.0,71402.0,7.77,1,-1
273,"""We believe that Marxism is an evil doctrine s...",2024-05-26 04:18:00,-0.8578,69222.0,68564.0,-0.95,0,-1
309,This past Thursday night was a defeat not only...,2024-06-29 20:54:00,-0.9924,60871.0,61526.0,1.08,1,-1
314,Andy Puzder just beat Never Trumper Charlie Ga...,2024-07-25 17:49:00,0.2540,64003.0,67550.0,5.54,1,-1


In [183]:
filtered_df.to_excel("analyse2.xlsx")

In [185]:
filtered_df.shape

(19, 8)

In [187]:
filtered_df[((filtered_df["sentiment_score"]>0)&(filtered_df["btc_price_increased"]==1))|((filtered_df["sentiment_score"]<0)&(filtered_df["btc_price_increased"]==0))].shape

(9, 8)